### Imports and model specifications

First let's make the official imports.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from syft.workers.node_client import NodeClient

And than those specific to PySyft, with one important note: **the local worker should not be a client worker.** *Non client workers can store objects and we need this ability to run a plan.*

In [2]:
import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch ie add extra functionalities 

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False


server = hook.local_worker

In [3]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [4]:
class Net2(torch.nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
alice = NodeClient(hook, "ws://localhost:6666" , id="alice")
bob = NodeClient(hook, "ws://localhost:6667" , id="bob")
charlie = NodeClient(hook, "ws://localhost:6668" , id="charlie")
grid = sy.PrivateGridNetwork(alice, bob, charlie)

ConnectionRefusedError: [Errno 111] Connection refused

In [6]:
net = Net()
net_2 = Net2()

In [7]:
net.build(torch.tensor([1., 2.]))

PlaceHolder[Id:33081020867]>tensor([-0.9411, -0.4946], grad_fn=<LogSoftmaxBackward>)

In [8]:
net.is_built

True

In [10]:
net.parameters()[0]

Parameter containing:
tensor([[-0.3980, -0.5149],
        [ 0.3790,  0.4471],
        [ 0.2829,  0.5795]], requires_grad=True)

In [16]:
params = list(net.parameters())

In [28]:
scalar = 2
with torch.no_grad():
    for para in net.parameters():
        para.set_(para.data * scalar)

In [29]:
net.parameters()

[Parameter containing:
 tensor([[ 2.0389,  1.5359],
         [ 0.7869, -2.0238],
         [-1.2199, -1.0579]], requires_grad=True),
 Parameter containing:
 tensor([-2.2338, -0.1276, -2.7893], requires_grad=True),
 Parameter containing:
 tensor([[-1.7107,  0.9264, -1.1322],
         [ 1.9440, -1.6527,  0.6478]], requires_grad=True),
 Parameter containing:
 tensor([0.4438, 1.8012], requires_grad=True)]

In [10]:
net_2 = Net()

In [11]:
net_2.build(torch.tensor([1., 2.]))

PlaceHolder[Id:78764901853]>tensor([-0.3031, -1.3414], grad_fn=<LogSoftmaxBackward>)

In [12]:
net_2.parameters()

[Parameter containing:
 tensor([[-0.5078,  0.5897],
         [ 0.4766,  0.3407],
         [-0.0497, -0.2112]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3358, -0.6308,  0.6308], requires_grad=True),
 Parameter containing:
 tensor([[ 0.1054,  0.2267, -0.5663],
         [-0.4486,  0.2008, -0.5370]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3283, -0.1429], requires_grad=True)]

In [10]:
fn = net.fix_precision()

In [12]:
fn.parameters()

[Parameter containing:
 (Wrapper)>FixedPrecisionTensor>tensor([[ -44, -483],
         [ 470, -344],
         [ -13,  419]]),
 Parameter containing:
 (Wrapper)>FixedPrecisionTensor>tensor([550, 283, -13]),
 Parameter containing:
 (Wrapper)>FixedPrecisionTensor>tensor([[ 482,  454,  443],
         [ 343,  -57, -282]]),
 Parameter containing:
 (Wrapper)>FixedPrecisionTensor>tensor([261, 361])]

In [13]:
enc_fn = fn.share(alice, bob, charlie)

/home/billy/billy_ws/fl_025/PySyft/syft/frameworks/torch/tensors/interpreters/additive_shared.py:79: UserWarning: Use dtype instead of field
  warnings.warn("Use dtype instead of field")
/home/billy/billy_ws/fl_025/PySyft/syft/frameworks/torch/tensors/interpreters/additive_shared.py:91: UserWarning: Default args selected
  warnings.warn("Default args selected")


In [10]:
ptr_net = net.send(alice)

In [12]:
ptr_net

[PointerPlan | me:94965250147 -> alice:97465553686]

In [11]:
res = alice.list_objects_remote()
print(res)

{14540616907: <Plan Plan id:14540616907 owner:alice Tags: #fss_eq_plan_1 built>
, 87699910351: <Plan Plan id:87699910351 owner:alice Tags: #fss_eq_plan_2 built>
, 38784523241: <Plan Plan id:38784523241 owner:alice Tags: #fss_comp_plan_1 built>
, 44535851286: <Plan Plan id:44535851286 owner:alice Tags: #fss_comp_plan_2 built>
, 43580050243: <Plan Plan id:43580050243 owner:alice Tags: #xor_add_1 built>
, 16476746363: <Plan Plan id:16476746363 owner:alice Tags: #xor_add_2 built>
, 77980299128: Parameter containing:
tensor([[ 0.6108, -0.5988],
        [ 0.5488,  0.2667],
        [ 0.6433, -0.3075]], requires_grad=True), 13652955971: Parameter containing:
tensor([-0.1835, -0.6214,  0.5547], requires_grad=True), 56414087678: Parameter containing:
tensor([[ 0.1457,  0.5677, -0.4467],
        [ 0.0672, -0.0787,  0.0578]], requires_grad=True), 84473430923: Parameter containing:
tensor([-0.2122,  0.0990], requires_grad=True), 97465553686: <Plan Net id:97465553686 owner:alice built>
}


In [13]:
res_t = alice.list_tensors_remote()

In [14]:
print(res_t)

{77980299128: Parameter containing:
tensor([[ 0.6108, -0.5988],
        [ 0.5488,  0.2667],
        [ 0.6433, -0.3075]], requires_grad=True), 13652955971: Parameter containing:
tensor([-0.1835, -0.6214,  0.5547], requires_grad=True), 56414087678: Parameter containing:
tensor([[ 0.1457,  0.5677, -0.4467],
        [ 0.0672, -0.0787,  0.0578]], requires_grad=True), 84473430923: Parameter containing:
tensor([-0.2122,  0.0990], requires_grad=True)}


In [15]:
ptr_net.parameters()

Websocket connection closed (worker: alice)
Created new websocket connection


RuntimeError: Websocket connection closed and creation of new connection failed.